In [1]:
import streamlit as st
import saspy

def filter_elements(callermsisdns):
    # Initialise une session SAS
    sas = saspy.SASsession(cfgname='httpsviya')
    
    for callermsisdn in callermsisdns:
        # Filtrage des éléments pour chaque callermsisdn
        sas.submit(f'''
            proc sql;
                create table dfra.localisation_switch_rest as
                select *
                from dfra.localisation_switch_
                where callermsisdn = '{callermsisdn}';
            quit;
        ''')
        # Ajoute un message pour indiquer que le filtrage est terminé
        st.write(f'Table filtrée pour callermsisdn {callermsisdn} envoyée vers SAS Visual Analytics')

    # Exécute des commandes SAS supplémentaires après le filtrage des données
    sas.submit('''
                /*****************************************************************************/
                /*  Start a session named mySession using the existing CAS server connection */
                /*  while allowing override of caslib, timeout (in seconds), and locale     */
                /*  defaults.                                                                */
                /*****************************************************************************/
                options cashost="bigdviyaccont1.orangemali.local" casport=5580;
                cas myCTsession sessopts=(caslib=casuser timeout=1800 locale="en_US");
                cas myCTsession list; 
                /*---------------------------------------------------------------------------------*/
                /*     Make all the defined (global) SAS Libraries available for this session      */
                /*---------------------------------------------------------------------------------*/
                caslib _all_ assign; 
                
                proc casutil incaslib="reportdfra" outcaslib="reportdfra";
                    droptable casdata="localisation_switch_rest" quiet;
                    load data=dfra.localisation_switch_rest casout="localisation_switch_rest" promote;
                    save casdata="localisation_switch_rest" replace;
                run;
    ''')

    # Affiche le lien vers SAS Visual Analytics
    st.write("Votre table filtrée a été envoyée vers SAS Visual Analytics.")
    st.write("Vous pouvez accéder au rapport ici :")
    url = "https://bigdataviya.orangemali.local/SASVisualAnalytics/?reportUri=%2Freports%2Freports%2F961b1c48-34ec-4d35-844c-097458c739e2&sectionIndex=0&objectName=vi95&quickView=true&sas-welcome=false"
    st.markdown(f"[Lien vers SAS Visual Analytics]({url})")

def main():
    st.title('Multi-Filtre App')
                    
    # Champ de saisie pour les callermsisdn
    callermsisdns_input = st.text_input('callermsisdn (séparés par des virgules):')
            
    # Bouton de filtrage
    if st.button('Filtrer'):
        callermsisdns = callermsisdns_input.split(',')
        filter_elements(callermsisdns)

if __name__ == '__main__':
    main()


2024-03-25 11:35:40.960 
  command:

    streamlit run C:\Users\diagne080894\Documents\Min\envs\pyenv1\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
